In [29]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression ,  Lasso
from sklearn.model_selection import cross_val_score, KFold,  GridSearchCV 
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from util.preprocess import preprocess

In [21]:
df_original = pd.read_csv('../data/train.csv')
df_test = pd.read_csv('../data/test.csv')
outpath = "../result/"

In [6]:
df_processed = preprocess(df_original)

In [8]:
df_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162691 entries, 0 to 162690
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   MONTH                162691 non-null  object 
 1   TOWN                 162691 non-null  object 
 2   FLAT_TYPE            162691 non-null  object 
 3   BLOCK                162691 non-null  object 
 4   STREET               162691 non-null  object 
 5   FLOOR_RANGE          162691 non-null  object 
 6   FLOOR_AREA_SQM       162691 non-null  float64
 7   FLAT_MODEL           162691 non-null  object 
 8   LEASE_COMMENCE_DATA  162691 non-null  int64  
 9   RESALE_PRICE         162691 non-null  float64
 10  RESALE_YEAR          162691 non-null  int64  
 11  RESALE_MONTH         162691 non-null  int64  
 12  FLAT_AGE             162691 non-null  int64  
 13  FLOOR_AVG            162691 non-null  int64  
dtypes: float64(2), int64(5), object(7)
memory usage: 17.4+ MB


In [52]:
# remove BLOCK and STREET as the result from LASSO
categorical_features = ['TOWN', 'FLAT_TYPE',  'FLAT_MODEL']
numerical_features = ['FLOOR_AREA_SQM', 'LEASE_COMMENCE_DATA', 
                      'RESALE_YEAR', 'RESALE_MONTH', 'FLAT_AGE', 'FLOOR_AVG']

In [53]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

### First we start with simple linear regression with the target variable (resale price) log transformed

In [54]:
# 5. Prepare data for the model and apply LOG transformation to the target
X = df_processed.drop('RESALE_PRICE', axis=1)
# Target variable is log-transformed due to right skewness
y = np.log(df_processed['RESALE_PRICE']) 

# 6. Build the full model pipeline (using Linear Regression)
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', LinearRegression())])

In [55]:
print("\nValidating model performance using 5-fold Cross-Validation...")

# Define CV method
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Calculate CV scores for required metrics
rmse_scores = np.sqrt(-cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=cv))
mae_scores = -cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv)
r2_scores = cross_val_score(model, X, y, scoring='r2', cv=cv)


Validating model performance using 5-fold Cross-Validation...


 Convert log-error metrics back to original price space for better interpretability
-  Note: This is an approximation since back-transforming MAE/RMSE from log space is complex.
 We will present the log-space metrics as standard practice for log-transformed targets.

In [56]:
print("\nCross-Validation Results (Metrics are calculated on Log-Transformed Price):")

# Primary Metric: RMSE
print(f"Primary Metric (RMSE): {rmse_scores.mean():.4f} (Avg across 5 folds)")

# Supporting Metrics: MAE and R-squared
print(f"Supporting Metric (MAE): {mae_scores.mean():.4f} (Avg across 5 folds)")
print(f"Supporting Metric (R-squared): {r2_scores.mean():.4f} (Avg across 5 folds)")


Cross-Validation Results (Metrics are calculated on Log-Transformed Price):
Primary Metric (RMSE): 0.1120 (Avg across 5 folds)
Supporting Metric (MAE): 0.0875 (Avg across 5 folds)
Supporting Metric (R-squared): 0.8945 (Avg across 5 folds)


In [57]:
#final model 
model.fit(X, y)

,steps,"[('preprocessor', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [58]:
# predict test data
new_data_processed = preprocess(df_test)

# Make the prediction (this predicts log(price))
predicted_log_price = model.predict(new_data_processed)

In [59]:
predicted_price = np.exp(predicted_log_price) 

In [60]:
predicted_price

array([451949.26096601, 585101.56846085, 470098.42285112, ...,
       571171.6764308 , 604515.28387047, 449606.81563555], shape=(50000,))

In [61]:
predicted_df = pd.DataFrame(data={'predicted price': predicted_price}, index=df_test.index)

In [62]:
predicted_df.reset_index().rename(columns={"index":"id","predicted price":"Predicted"}) \
    .to_csv(outpath+"submission_baseline1.csv",header=True, index=False)

## Next we try to use a LASSO regression, with grid search on its parameters

In [63]:
# Build the initial pipeline (Lasso needs to be named 'regressor' to match the param_grid)
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', Lasso(random_state=42))])

# Define the parameter grid for alpha tuning
# We will search over a range of common alpha values for Lasso
param_grid = {
    'regressor__alpha': [ 0.001, 0.01, 0.1, 1.0, 10.0, 100]
}

# Define CV method (5-fold)
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Set up GridSearchCV
# We use 'neg_mean_squared_error' because GridSearchCV maximizes the score, 
# and minimizing MSE is equivalent to maximizing negative MSE.
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=cv,
    scoring='neg_mean_squared_error',
    n_jobs=-1, # Use all available cores
    verbose=1
)

In [64]:
grid_search.fit(X, y)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


,estimator,Pipeline(step...m_state=42))])
,param_grid,"{'regressor__alpha': [0.001, 0.01, ...]}"
,scoring,'neg_mean_squared_error'
,n_jobs,-1
,refit,True
,cv,KFold(n_split... shuffle=True)
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('num', ...), ('cat', ...)]"


In [65]:
best_model = grid_search.best_estimator_
best_alpha = grid_search.best_params_['regressor__alpha']
best_rmse_cv = np.sqrt(-grid_search.best_score_) # Convert negative MSE back to RMSE
print(f"\nOptimal Lasso Alpha found: {best_alpha}")


Optimal Lasso Alpha found: 0.001


In [66]:
mae_scores = -cross_val_score(best_model, X, y, scoring='neg_mean_absolute_error', cv=cv)
r2_scores = cross_val_score(best_model, X, y, scoring='r2', cv=cv)

print("\nCross-Validation Results (Metrics are calculated on Log-Transformed Price using optimal Lasso):")

# Primary Metric: RMSE
print(f"Primary Metric (RMSE): {best_rmse_cv:.4f} (from GridSearchCV)")
# Supporting Metrics: MAE and R-squared
print(f"Supporting Metric (MAE): {mae_scores.mean():.4f} (Avg across 5 folds)")
print(f"Supporting Metric (R-squared): {r2_scores.mean():.4f} (Avg across 5 folds)")


Cross-Validation Results (Metrics are calculated on Log-Transformed Price using optimal Lasso):
Primary Metric (RMSE): 0.1207 (from GridSearchCV)
Supporting Metric (MAE): 0.0930 (Avg across 5 folds)
Supporting Metric (R-squared): 0.8773 (Avg across 5 folds)


In [67]:
# Make the prediction (this predicts log(price))
predicted_log_price = best_model.predict(new_data_processed)

In [68]:
predicted_price = np.exp(predicted_log_price) 
predicted_df = pd.DataFrame(data={'predicted price': predicted_price}, index=df_test.index)
predicted_df.reset_index().rename(columns={"index":"id","predicted price":"Predicted"}) \
    .to_csv(outpath+"submission_lasso.csv",header=True, index=False)

In [69]:
print("\n--- Feature Coefficients (Lasso Feature Selection) ---")

# 1. Get feature names from the preprocessor's transformers
preprocessor = best_model.named_steps['preprocessor']

# Get names of all transformed features: numerical (scaled) + categorical (one-hot encoded)
# Note: get_feature_names_out() automatically includes prefixes like 'cat__TOWN_'
cat_feature_names = list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features))
all_feature_names = numerical_features + cat_feature_names

# 2. Get the coefficients from the best Lasso model
lasso_coefficients = best_model.named_steps['regressor'].coef_

# 3. Create a DataFrame to inspect coefficients
coeff_df = pd.DataFrame({
    'Feature': all_feature_names,
    'Coefficient': lasso_coefficients
}).sort_values(by='Coefficient', key=abs, ascending=False)

print(f"\nTotal features used: {len(coeff_df)}")
# Use a small tolerance (1e-6) for "zero" due to floating point arithmetic
zero_coeffs_count = (coeff_df['Coefficient'].abs() < 1e-6).sum()
print(f"Features whose coefficient was set to zero by Lasso: {zero_coeffs_count}")


--- Feature Coefficients (Lasso Feature Selection) ---

Total features used: 60
Features whose coefficient was set to zero by Lasso: 25


In [70]:
print(coeff_df[:50])
# 1st round: we observe that BLOCK and STREET are both irrelevant
# 2nd round: we can potentially group flat model types that are set to 0 with to "others" to reduce noise

                              Feature  Coefficient
0                      FLOOR_AREA_SQM     0.229221
14                 TOWN_choa chu kang    -0.228202
30                     TOWN_woodlands    -0.221134
25                     TOWN_sembawang    -0.190834
10                   TOWN_bukit merah     0.188555
24                    TOWN_queenstown     0.183145
13                  TOWN_central area     0.180060
19                   TOWN_jurong west    -0.177963
26                      TOWN_sengkang    -0.159397
8                         TOWN_bishan     0.159194
11                 TOWN_bukit panjang    -0.150251
31                        TOWN_yishun    -0.138305
2                         RESALE_YEAR     0.134414
21                 TOWN_marine parade     0.130659
1                 LEASE_COMMENCE_DATA     0.126991
23                       TOWN_punggol    -0.126638
43                    FLAT_MODEL_dbss     0.124014
20               TOWN_kallang/whampoa     0.117731
9                    TOWN_bukit